In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import spatial
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot
from IPython.display import HTML
from sklearn.manifold import TSNE
from matplotlib import animation

In [34]:
macros = pd.read_excel('Macros.xlsx')
categories = pd.read_excel('DACC_Hackathon_Categories_Information.xlsx')

In [36]:
!pip install sentence-transformers
!pip install scipy
!pip install nltk
import nltk
import scipy
nltk.download('punkt')
from sentence_transformers import SentenceTransformer
#Loading Pre-trained BERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
items = list(categories['Category'])

##**Recommending healthier choices**

In [38]:
def check_healthierV2(item1, item2, cal1, cal2):
  #We give the trans-fat categorical variable a weight of 10
  weight = 10
  score = 0

  for i in range(len(item1[:2])):
    #we reward nutritional value for higher wholegrains and calcium
    score += (item1[i] - item2[i])

  
  for i in range(len(item1[2:5])):
    #we penalise heavier for food with high sodium, saturated fats and sugar
    score -= (item1[i] - item2[i])*weight

  #we penalise the food if it contains trans-fat
  score += (item1[5] - item2[5])*weight

  #we penalise the food if it has higher calories
  score -= (cal1 - cal2)/10

  
  if score >= 0:
    return True
  else:
    return False

In [39]:
def similar_food(macros, items):
  food_dict = {}

  for i in range(len(items)):
    o_item = model.encode(items[i])
    temp = []
    print(f'Finding similar food for {items[i]}')
    for j in range(len(items)):
      n_item = model.encode(items[j])
      distance = scipy.spatial.distance.cdist([o_item], [n_item], "cosine")[0][0]
      temp.append(distance)
    #Top 10 semantically similar food
    temp = np.argsort(temp)[:10]
    #print(temp)
    food_dict[macros.Category[i]] = list(macros.Category[temp][1:])

  return food_dict

In [40]:
def list_reco(food_dict, macros, categories):

  dict_foodgrps = {}

  for i in range(len(macros)):
    if categories.FoodGrp[i] not in dict_foodgrps:
      dict_foodgrps[categories.FoodGrp[i]] = []
  
    dict_foodgrps[categories.FoodGrp[i]].append(macros.Category[i])

  list_of_reco = {}
  for k, v in food_dict.items():
    fg = dict_foodgrps[categories[categories['Category'] ==  k].FoodGrp.values[0]]
    list_of_reco[k] = []
    for item in v:
      if item in fg:
        list_of_reco[k].append(item)

  
  return list_of_reco

In [41]:
def main(macros, items, categories):
  
  food_dict = similar_food(macros, items)
  list_of_reco = list_reco(food_dict, macros, categories)

  final_reco = {}

  #print(list_of_reco)
  print('Looking for healthier options:')
  print('------------------------------------------------------------------------------')
  for k,v in list_of_reco.items():
    if not v:
      continue
    o_mac = macros[macros['Category'] == k].iloc[:, 3:].values[0]
    o_cal = macros[macros['Category'] == k]['Calories/100g'].values[0]
  
    lowest_mac = o_mac
    lowest_cal = o_cal

    for item in v:
      i_mac = macros[macros['Category'] == item].iloc[:, 3:].values[0]
      i_cal = macros[macros['Category'] == item]['Calories/100g'].values[0]
      #if substitute is a healthier choice than the original
      if not check_healthierV2(lowest_mac, i_mac, lowest_cal, i_cal):
        final_reco[k] = item
        break

  print('\n')
  print('Churning healther recommendations:')

  print('------------------------------------------------------------------------------')

  for k, v in final_reco.items():
    if not v:
      continue
    print(f'For {k}, we recommend {v}')

  print('------------------------------------------------------------------------------')

  return final_reco

###**Running the main reccomendation code**

In [42]:
final_reco = main(macros, items, categories)

Finding similar food for Beer
Finding similar food for Belacan
Finding similar food for Bird Nest
Finding similar food for Biscuits
Finding similar food for Bouilon
Finding similar food for Butter
Finding similar food for Cake
Finding similar food for Canned Product
Finding similar food for Cereal Beverage
Finding similar food for Cereals
Finding similar food for Cheese
Finding similar food for Chicken Essence
Finding similar food for Nut Spread
Finding similar food for Chocolate
Finding similar food for Coconut Milk
Finding similar food for Coffee
Finding similar food for Condensed Milk
Finding similar food for Confectionery
Finding similar food for Cooking Oils
Finding similar food for Cooking Sauces
Finding similar food for Cordials
Finding similar food for Creamer
Finding similar food for Carbonated Soft Drinks
Finding similar food for Cultured Milk
Finding similar food for Drinking Water
Finding similar food for Eggs
Finding similar food for Energy Drinks
Finding similar food for 

##**Demo On More Specific Food (In a bigger market)**

In [43]:
demo = pd.read_excel('Macros.xlsx', sheet_name='Demo')

In [44]:
demo_items = list(demo.Food.values)

In [45]:
def check_healthier_demo(item1, item2, cal1, cal2):
  #We give the trans-fat categorical variable a weight of 10
  weight = 10
  score = 0

  for i in range(len(item1[:2])):
    #we reward nutritional value for higher wholegrains and calcium
    score += (item1[i] - item2[i])

  
  for i in range(len(item1[2:5])):
    #we penalise heavier for food with high sodium, saturated fats and sugar
    score -= (item1[i] - item2[i])

  #we penalise the food if it contains trans-fat
  score += (item1[5] - item2[5])*weight

  #we penalise the food if it has higher calories
  score -= (cal1 - cal2)/10

  
  if score >= 0:
    return True
  else:
    return False

In [46]:
def demo_similar_food(demo, demo_items):
  demo_food_dict = {}

  for i in range(len(demo_items)):
    o_item = model.encode(demo_items[i])
    temp = []
    print(f'Finding similar food for {demo_items[i]}')
    for j in range(len(demo_items)):
      n_item = model.encode(demo_items[j])
      distance = scipy.spatial.distance.cdist([o_item], [n_item], "cosine")[0][0]
      temp.append(distance)
    #Top 3 semantically similar food
    temp = np.argsort(temp)[:10]
    #print(temp)
    demo_food_dict[demo.Food[i]] = list(demo.Food[temp][1:])

  return demo_food_dict

In [47]:
def demo_list_reco(demo_food_dict, demo):

  demo_dict_foodgrps = {}

  for i in range(len(demo)):
    if demo.Category[i] not in demo_dict_foodgrps:
      demo_dict_foodgrps[demo.Category[i]] = []
  
    demo_dict_foodgrps[demo.Category[i]].append(demo.Food[i])


  demo_list_of_reco = {}
  for k, v in demo_food_dict.items():
    fg = demo_dict_foodgrps[demo[demo['Food'] ==  k].Category.values[0]]
    demo_list_of_reco[k] = []
    for item in v:
      if item in fg:
        demo_list_of_reco[k].append(item)
  
  return demo_list_of_reco

In [48]:
def demo_main(demo, demo_items):
  
  demo_food_dict = demo_similar_food(demo, demo_items)
  demo_list_of_reco = demo_list_reco(demo_food_dict, demo)

  demo_final_reco = {}

  print('Looking for healthier options:')
  print('------------------------------------------------------------------------------')
  for k,v in demo_list_of_reco.items():
    if not v:
      continue
    o_mac = demo[demo['Food'] == k].iloc[:, 3:].values[0]
    o_cal = demo[demo['Food'] == k]['Calories'].values[0]
  
    best_sub = ''
    lowest_mac = o_mac
    lowest_cal = o_cal

    for item in v:
      i_mac = demo[demo['Food'] == item].iloc[:, 3:].values[0]
      i_cal = demo[demo['Food'] == item]['Calories'].values[0]
      #if substitute is a healthier choice than the original
      if not check_healthier_demo(lowest_mac, i_mac, lowest_cal, i_cal):
        demo_final_reco[k] = item
        break

  print('\n')
  print('Churning healther recommendations in a bigger market (Future Recommendations):')

  print('------------------------------------------------------------------------------')

  for k, v in demo_final_reco.items():
    if not v:
      continue
    print(f'For {k}, we recommend {v}')

  print('------------------------------------------------------------------------------')

  return demo_final_reco

###**From here, we can see that with more specific items, we are able to recommend better and similar healthier substitutes.**

In [49]:
demo_final_reco = demo_main(demo, demo_items)

Finding similar food for Vegetable Oil
Finding similar food for Olive Oil
Finding similar food for Avocado Oil
Finding similar food for White Rice
Finding similar food for Brown Rice
Finding similar food for Qunoia Rice
Finding similar food for Spaghetti
Finding similar food for Milk
Finding similar food for Oat Milk
Finding similar food for Almond Milk
Finding similar food for Chocolate Milk
Looking for healthier options:
------------------------------------------------------------------------------


Churning healther recommendations in a bigger market (Future Recommendations):
------------------------------------------------------------------------------
For Vegetable Oil, we recommend Olive Oil
For Avocado Oil, we recommend Vegetable Oil
For White Rice, we recommend Brown Rice
For Qunoia Rice, we recommend Brown Rice
For Spaghetti, we recommend White Rice
For Milk, we recommend Oat Milk
For Oat Milk, we recommend Almond Milk
For Chocolate Milk, we recommend Almond Milk
------------

## **Evaluation**
Assuming 20% of the customers are receptive towards the recommendations, we calculate the percentage reduction in sugar, sodium and saturated fats and increase in fibre and calcium if they had taken our recommendation.

In [50]:
txn = pd.read_csv('DACC_Hackathon_TransactionData.csv')
demog = pd.read_excel('DACC_Hackathon_Panelists_Demogs.xlsx')
cluster = pd.read_excel('cluster.xlsx')

macros.Category = macros.Category.str.upper()

FileNotFoundError: ignored

In [ ]:
txn = txn.rename(columns={"Panel ID":"ID"})
txn.ID.astype(int)
txn['Pack Size'].astype(int)

In [ ]:
df = pd.merge(txn, macros, on='Category', how='left')
df = pd.merge(df, cluster, on='ID', how='left')

In [ ]:
# Since Clusters 2 and 3 has the highest proportion of unhealthy individuals,
# we want to target these two groups

target = df[(df.Clusters == 2)|(df.Clusters == 3)]
target = target.drop(['Date', 'Pack Size', 'Volume', 'Spend', 'Calories/100g', 'Price per Volume'], axis=1)

In [ ]:
import random
random.seed(3101)
flag = pd.DataFrame(random.choices([0,1], weights=(0.8,0.2), k=target.ID.nunique()))
cust = pd.DataFrame(target.ID.unique())

cust_flag = pd.concat([cust, flag], axis=1)
cust_flag = cust_flag.set_axis(['ID', 'flag'], axis=1, inplace=False)

# We only examine the 20% of customers that take our recommendation
target = pd.merge(target, cust_flag, on='ID', how='left')
target = target[target.flag == 1]

In [ ]:
target

In [ ]:
target2 = target.copy()
target2 = target2[['ID', 'Category']]

In [ ]:
def take_recommendation(items):
  
  new_items = []
  
  for item in items:
    item = str.title(item)
    if item in final_reco:
      new_items.append(final_reco[item].upper())
    else:
      new_items.append(item.upper())
    
  return new_items

In [ ]:
target2.Category = take_recommendation(target2.Category)
target2 = pd.merge(target2, macros, on='Category', how='left')
target2 = target2.drop(['Calories/100g', 'Price per Volume'], axis=1)

In [ ]:
target2

In [ ]:
# Finding the percentage increase/decrease in the five different indicators
(target2.sum()[5:10] - target.sum()[5:10])/target.sum()[5:10] * 100

From the results above, we see that 20% of the clusters take up the recommendation and use the recommended items, sugar intake would be cut by about 32% and saturated fats intake would be cut by 64%.

Since majority of the unhealthy individuals from clusters 2 and 3 are either "Obese" or "Over Weight", by cutting the amount of sugar and saturated fat consumed by these clusters will likely bring about a positive effect on the BMI, bringing these people down towards the "Healthy" category.